# Семинар 17.

# Линейное программирование. Симплекс-метод

## Текущее место в программе

- Безусловная оптимизация
    - ~~Одномерная минимизация~~
    - ~~Градиентный спуск~~
    - ~~Метод Ньютона и квазиньютоновские методы~~
    - ~~Метод сопряжённых градиентов~~
- Условная оптимизация
    - <span style="color:red">Линейное программирование: симплекс-метод</span>
    - Линейное программирование: прямо-двойственный метод
    - Методы проекции градиента и условного градиента
    - Методы штрафов и барьеров
    - Метод модифицированной функции Лагранжа
    - Последовательное квадратичное программирование
- Разные темы
    - ~~Задача наименьших квадратов~~
    - Проксимальные методы
    - Оптимальные методы и нижние оценки
    - Зеркальный спуск
    - Обзор стохастических методов

## Постановка задачи
Даны векторы $x, c \in \mathbb{R}^n$, $b \in \mathbb{R}^m$ и матрица $A \in \mathbb{R}^{m \times n}$ такая что $m < n$ и $\mathrm{rank}(A) = m$ 
- Стандартная форма
\begin{equation*}
\begin{split}
&\min_x c^{\top}x \\
\text{s.t. } & Ax \leq b\\
& x_i \geq 0, \; i = 1,\dots, n
\end{split}
\end{equation*}
- Каноническая форма (везде далее рассматривается именно она)
\begin{equation*}
\begin{split}
&\min_x c^{\top}x \\
\text{s.t. } & Ax = b\\
& x_i \geq 0, \; i = 1,\dots, n
\end{split}
\end{equation*}

### Преобразование постановок

- $Ax = b \equiv 
\begin{cases}
Ax \leq b\\
Ax \geq b
\end{cases}
$
- $Ax \leq b \equiv 
\begin{cases}
Ax + z =  b\\
z \geq 0
\end{cases}$ 
- свободная переменная $x \equiv 
\begin{cases}
x = u - v\\
u \geq 0, \; v \geq 0
\end{cases}$

## Приложения

- Производство оптимального количества товара при ресурсных ограничениях:
\begin{equation*}
\begin{split}
&\max_x c^{\top}x \\
\text{s.t. } & Ax \leq b\\
& x_i \geq 0, \; i = 1,\dots, n,
\end{split}
\end{equation*}
где $x_i$ - количество $i$-го товара, $c_i$ - прибыль от производства $i$-го товара, $b_i$ - доступное количество $i$-го материала, $a_{ij}$ - количество $i$-го материала, который требуется для производства единицы $j$-го товара.

- Потоки в сетях: транспортная задача, задача о максимальном потоке, выбор пути в коммуникационной сети для передачи сообщения с минимальной стоимостью

- Задача регрессии в нормах $\ell_1$ и $\ell_{\infty}$ может быть сведена к задаче линейного программирования

## Какой ответ может дать алгоритм?

- Задача разрешима и точка минимума $x^*$
- Задача неограничена и минимальное значение равно $-\infty$
- Задача неразрешима, то есть допустимое множество пусто

## Предварительные соображения

- Задача линейного программирования - это поиск минимума линейной функции на многомерном многоугольнике

**Вопросы:**

- при каких условиях есть решение?
- среди каких точек искать решение?
- как именно искать решение?

## Определения

1. Tочка из допустимого множества называется *вершиной многоугольника*, если она не лежит на отрезке между двумя другими точками многоугольника.
2. Точка $x$ называется *угловой точкой* многоугольника, если
    - она лежит в множестве
    - существуте такое множество $\mathcal{B} \subset \{1, \dots, n \}$, что 
        - $|\mathcal{B}| = m$
        - $i \notin \mathcal{B} \Rightarrow x_i = 0$
        - матрица $B = [a_i]_{i \in \mathcal{B}}$ невырождена, где $a_i$ - $i$-ый столбец матрицы $A$. Матрица $B$ называется *матрицей базиса*

**Теорема** Все угловые точки являются вершинами соответствующего многоугольника.

## Фундаментальная теорема линейного программирования

1. Если в задаче линейного програмирования допустимое множество непусто, тогда оно имеет как минимум одну угловую точку.
2. Если задача линейного программирования имеет решения, тогда хотя бы одно из них является угловой точкой.
3. Если задача линейного программирования ограничена и допустимое множество непусто, тогда она имеет оптимальное решение.

## Симплекс-метод

- Симплекс-метод [предложил](https://www.rand.org/pubs/reports/R366.html) Дж. Данциг в 1947.
- Его появление совпало с развитием компьютеров и явилось одним из первых доказательств их полезности
- Имел большой успех в связи с многочисленными приложениями в экономике и планировании производства
- Включён в [список](http://www.uta.edu/faculty/rcli/TopTen/topten.pdf) лучших алгоритмов XX века по версии редакторов SIAM

### Общая схема

1. Находим произвольную угловую точку допустимого множества
2. Переходим в сопряжённую ей угловую точку так, чтобы целевая функция уменьшалась
3. Если можно дальше уменьшить значение целевой функции, переходим в следующую угловую точку

**Вопрос:** как формализовать и выполнить эти шаги?

## Псевдокод итерации симплекс-метода

Дана угловая точка $x$, соответствующая ей матрица базиса $B$ и множество индексов $\mathcal{B}$.

1. Вычислить *оценки замещения* (reduced costs) $\overline{c}_j = c_j - c^{\top}_{\mathcal{B}}B^{-1}a_j$ для всех $j \not\in \mathcal{B}$.
    - если $\overline{c}_j \geq 0$ для всех $j$, то текущее значение является оптимальным и уменьшить целевую функцию нельзя
    - иначе <span style="color:red"> выбрать </span> индекс $j^*$, для которого $\overline{c}_{j^*} < 0$
2. Вычислить $u = B^{-1}a_{j^*}$.
    - если все компоненты $u$ неположительны, то задача неограничена, оптимальное значение равно $-\infty$
    - если есть положительные компоненты, то 
    $$
    \theta^* = \min_{\{i | u_i > 0\}} \frac{x_{\mathcal{B}(i)}}{u_i}
    $$
3. Пусть $\ell$ <span style="color:red"> такой </span> индекс, что 
    $$
    \theta^* = \frac{x_{\mathcal{B}(\ell)}}{u_{\ell}}.
    $$
    Формируем новую матрицу базиса $\hat{B}$ с помощью замены столбца $a_{\mathcal{B}(\ell)}$ на столбец $a_{j^*}$. Новая угловая точка $\hat{x}$, соответствующая матрице базиса $\hat{B}$ определяется так
    $$
    \hat{x}_k = 
    \begin{cases}
    \theta^*,& \text{если } k = j^*\\
    x_{\mathcal{B}(k)} - \theta^*u_k & \text{если } k \neq \ell
    \end{cases}
    $$

## Существующие реализации симплекс-метода

- Наивная реализация
- <span style="color:red"> Табличная реализация </span>
- Модифицированная реализация

### Детали


## Проблема

Если одна из угловых точек является *вырожденной*, то это может привести к **зацикливанию** симплекс-метода!

## Как искать начальную угловую точку?

- Двухфазный симплекс-метод
- M-метод

## Пример использования

In [16]:
import scipy.optimize as scopt
import numpy as np
n = 1000
m = 10
c = 10 * np.random.rand(n)
b = np.random.rand(m)
A = np.random.randn(m, n)
res = scopt.linprog(c, A, b, bounds=[(1, None) for i in xrange(n)])
print(res)

     fun: 198.00000000000063
 message: 'Optimization failed. Unable to find a feasible starting point.'
     nit: 1000
  status: 2
 success: False
       x: nan


## Сложность

- Было показано, что в худшем случае время работы симплекс-метода <span style="color:red"> **экспоненциально**</span> зависит от размерности задачи!
- Однако на практике сложность чаще всего пропорциональна количеству ограничений и симплекс-метод сходится быстро
- Почему это так не ясно до сих пор

### Пример Klee, Minty (1972) [[1]](https://en.wikipedia.org/wiki/Klee–Minty_cube)

В задаче
\begin{equation*}
\begin{split}
& \min_x 2^{n-1}x_1 + 2^{n-2}x_2 + \dots + 2x_{n-1} + x_n\\
\text{s.t. } & x_1 \leq 5\\
& 4x_1 + x_2 \leq 25\\
& 8x_1 + 4x_2 + x_3 \leq 125\\
& \ldots\\
& 2^n x_1 + 2^{n-1}x_2 + 2^{n-2}x_3 + \ldots + x_n \leq 5^n\\
& x \geq 0
\end{split}
\end{equation*}
начиная с точки $x_0 = 0$ и следуя симплекс-методу, необходимо обойти $2^n - 1$ вершину.

**Упражнение**: решите эту задачу для $n = 2$ и $n = 3$, и обобщите результат, получив решение для произвольного $n$.

## Резюме

- Задача линейного программирования
- Приложения
- Симплекс-метод